In [ ]:
#@title *Prepare* { display-mode: "form", run: "auto" }

#@markdown # **Prepare the environment**
#@markdown Execute to install necessary packages, modules, fonts

#@markdown Mount Google Drive at /content/drive (your drive folder at /content/drive/MyDrive):
mount_gdrive = True #@param {type:"boolean"}
#@markdown Force mount again. Useful for bug cases:
force_remount = False #@param {type:"boolean"}

# remove the old log
!rm -rf error_log.txt > /dev/null

import os
import subprocess
from google.colab import drive

# mount Google Drive
if mount_gdrive:
    print("Mounting your Google Drive | Waiting user Allow Access | ", end='')
    try:
        drive.mount('/content/drive/', force_remount=force_remount)
    except Exception as e:
        print(f"[✗]: {e}")

# update code
print("Ensuring the LeGen code is existing and updated...", end='')
repo_url = "https://github.com/matheusbach/legen.git"
local_folder = "/content/src"  # LeGen source path

# Create directory if it does not exist
os.makedirs(local_folder, exist_ok=True)

# Try git status in the directory
git_task = "git fetch"
process = subprocess.Popen(git_task, cwd=local_folder, shell=True)
return_code = process.wait()
if return_code == 0:
  git_task = "git fetch && git reset --hard origin/main && git pull"
else:
  !rm -rf local_folder > /dev/null
  os.makedirs(local_folder, exist_ok=True)
  git_task = f"git clone {repo_url} {local_folder}"

# If it is a git repo, fetch, reset, and pull. Else, clone.
with open('/content/error_log.txt', 'a') as f:
    process = subprocess.Popen(git_task, cwd=local_folder, shell=True, stderr=f)
    return_code = process.wait()
    print("[✔]" if return_code == 0 else "[✗]")

# install pip requirements.txt updating
print("Installing or updating pip requirements...", end='')
with open('/content/error_log.txt', 'a') as f:
    process = subprocess.Popen('pip3 install --upgrade $(grep -v "git+https://github.com/matheusbach/whisperx.git" requirements.txt | grep -v "torch") && pip3 install git+https://github.com/matheusbach/whisperx.git --upgrade', cwd=local_folder, shell=True, stderr=f)
    return_code = process.wait()
    print("[✔]" if return_code == 0 else "[✗]")

# install ffmpeg and xvfb
print("Installing FFmpeg and xvfb...", end='')
with open('/content/error_log.txt', 'a') as f:
    process = subprocess.Popen('apt update -y ; apt install ffmpeg xvfb -y', shell=True, stderr=f)
    return_code = process.wait()
    print("[✔]" if return_code == 0 else "[✗]")

# install pip requirements.txt updating
print("Installing fonts...", end='')
with open('/content/error_log.txt', 'a') as f:
    process = subprocess.Popen('echo ttf-mscorefonts-installer msttcorefonts/accepted-mscorefonts-eula select true | debconf-set-selections && apt install -y ttf-mscorefonts-installer && fc-cache -f -v', shell=True, stderr=f)
    return_code = process.wait()
    print("[✔]" if return_code == 0 else "[✗]")

# create a virtual display
os.system('Xvfb :1 -screen 0 2560x1440x8  &') # create virtual display with size 1600x1200 and 8 bit color. Color can be changed to 24, 16 or 8
os.environ['DISPLAY'] = ':1.0' # tell X clients to use our virtual DISPLAY :1.0.

print("\nPreparation tasks done.")

In [ ]:
#@title *Configure* { display-mode: "form", run: "auto" }
#@markdown # **Define Software Settings**
#@markdown ---
#@markdown ## General Options
#@markdown Set where your files are located (your Drive is the base /content/drive/MyDrive):
import os
from IPython.display import display
import ipywidgets as widgets
input_path = "/content/drive/MyDrive/LeGen/media" #@param {type:"string"}
#@markdown Set where output soft subs (.srt and embed) videos will be saved on your Drive:
output_softsubs_path = "/content/drive/MyDrive/LeGen/softsubs" #@param {type:"string"}
#@markdown Set where output hard subs (burned in) videos will be saved on your Drive:
output_hardsubs_path = "/content/drive/MyDrive/LeGen/hardsubs" #@param {type:"string"}

#@markdown ---
overwrite = False #@param {type:"boolean"}
normalize = False #@param {type:"boolean"}
copy_extra_files = True #@param {type:"boolean"}

# generate_srt_files = True #@param {type:"boolean"}
generate_softsub = True #@param {type:"boolean"}
generate_hardsub = True  # @param {type:"boolean"}

# buttons for directory create
button = widgets.Button(description="Create directories!")
output = widgets.Output()


def on_button_clicked(b):
  # Create input and output directories if it does not exist
  os.makedirs(input_path, exist_ok=True)
  if generate_softsub:
    os.makedirs(output_softsubs_path, exist_ok=True)
  if generate_hardsub:
    os.makedirs(output_hardsubs_path, exist_ok=True)
  with output:
    print("Done.")


button.on_click(on_button_clicked)
display(button, output)

#@markdown ---
#@markdown ## Transcription Settings:
transcription_engine = 'WhisperX' # @param ["Whisper", "WhisperX"]
transcription_device = 'auto' #@param ["auto", "cpu", "cuda"]
transcription_model = 'large' #@param ["tiny", "small", "medium", "large", "large-v1", "large-v2", "large-v3"]
compute_type = 'default' # @param ["default", "int8", "int16", "float16", "float32"]
batch_size = 12 # @param {type: "number"}
transcription_input_lang = 'auto detect' #@param ["auto detect", "af", "am", "ar", "as", "az", "ba", "be", "bg", "bn", "bo", "br", "bs", "ca", "cs", "cy", "da", "de", "el", "en", "es", "et", "eu", "fa", "fi", "fo", "fr", "gl", "gu", "ha", "haw", "he", "hi", "hr", "ht", "hu", "hy", "id", "is", "it", "ja", "jw", "ka", "kk", "km", "kn", "ko", "la", "lb", "ln", "lo", "lt", "lv", "mg", "mi", "mk", "ml", "mn", "mr", "ms", "mt", "my", "ne", "nl", "nn", "no", "oc", "pa", "pl", "ps", "pt", "ro", "ru", "sa", "sd", "si", "sk", "sl", "sn", "so", "sq", "sr", "su", "sv", "sw", "ta", "te", "tg", "th", "tk", "tl", "tr", "tt", "uk", "ur", "uz", "vi", "yi", "yo", "zh"]

#@markdown ---
#@markdown ## Translation Settings:
#@markdown Set the destination langrage code. Set to same as original to skip translation
target_language_code = 'pt' #@param ["af", "sq", "am", "ar", "hy", "as", "ay", "az", "bm", "eu", "be", "bn", "bho", "bs", "bg", "ca", "ceb", "zh-CN", "zh-TW", "co", "hr", "cs", "da", "dv", "doi", "nl", "en", "eo", "et", "ee", "fil", "fi", "fr", "fy", "gl", "ka", "de", "el", "gn", "gu", "ht", "ha", "haw", "he", "hi", "hmn", "hu", "is", "ig", "ilo", "id", "ga", "it", "ja", "jv", "kn", "kk", "km", "rw", "gom", "ko", "kri", "ku", "ckb", "ky", "lo", "la", "lv", "ln", "lt", "lg", "lb", "mk", "mai", "mg", "ms", "ml", "mt", "mi", "mr", "mni-Mtei", "lus", "mn", "my", "ne", "no", "ny", "or", "om", "ps", "fa", "pl", "pt", "pa", "qu", "ro", "ru", "sm", "sa", "gd", "nso", "sr", "st", "sn", "sd", "si", "sk", "sl", "so", "es", "su", "sw", "sv", "tl", "tg", "ta", "tt", "te", "th", "ti", "ts", "tr", "tk", "ak", "uk", "ur", "ug", "uz", "vi", "cy", "xh", "yi", "yo", "zu"]

#@markdown ---
#@markdown ## Video Settings:
codec_video = "h264"  #@param ["h264", "hevc", "mpeg4"]
video_hardware_api = "auto"  #@param ["auto", "none", "nvenc", "vaapi", "amf", "v4l2m2m", "qsv", "videotoolbox", "cuvid"]

codec_audio = "aac"  #@param ["aac", "libopus", "libmp3lame", "pcm_s16le"]

In [ ]:
#@title *Run* { display-mode: "form" }
#@markdown # **Run LeGen.py**

print("Starting LeGen...")
import torch
try:
    import tensorflow  # required in Colab to avoid protobuf compatibility issues
except ImportError:
    pass

# Create input and output directories if it does not exist
os.makedirs(input_path, exist_ok=True)
if generate_softsub:
  os.makedirs(output_softsubs_path, exist_ok=True)
if generate_hardsub:
  os.makedirs(output_hardsubs_path, exist_ok=True)

#build query
query = f" -i '{input_path}'"
query += f" --output_softsubs '{output_softsubs_path}'"
query += f" --output_hardsubs '{output_hardsubs_path}'"
query += " --overwrite" if overwrite else ""
query += " --norm" if normalize else ""
query += " --copy_files" if not copy_extra_files else ""
query += " --disable_softsubs" if not generate_softsub else ""
query += " --disable_hardsubs" if not generate_hardsub else ""
query += f" -ts:e {transcription_engine.lower()}"
query += f" -ts:d {transcription_device.lower()}"
query += f" -ts:m {transcription_model}"
query += f" -ts:c {compute_type}"
query += f" -ts:b {batch_size}"
query += f" --input_lang {transcription_input_lang}" if transcription_input_lang != "auto detect" else ""
query += f" --translate {target_language_code.lower()}"
query += f" -c:v {codec_video}" + ("" if video_hardware_api == "none" else f"_{video_hardware_api}" if video_hardware_api != "auto" else "_nvenc" if torch.cuda.is_available() else "")
query += f" -c:a {codec_audio}"

#run python script
print(f"command line: python3 /content/src/legen.py {query}", end="\n\n")
!python3 /content/src/legen.py $query